In [1]:
import pandas as pd
import tweepy
from textblob import TextBlob


In [2]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [35]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

#polarities = {1:'Very Negative',2:'Negative', 3:'Neutral', 4:'Positive', 5:'Very Positive'}


emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

emoticons = emoticons_happy.union(emoticons_sad)



def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    emoji_pattern = re.compile("["  
         u"\U0001F600-\U0001F64F" # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #tweet = re.sub(r'@','',tweet)
    
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation and word not in emoticons):  # remove punctuation and emoticons
            tweets_clean.append(word)
        
        if word in emoticons_happy:
            tweets_clean.append('Happy')
        elif word in emoticons_sad:
            tweets_clean.append('Sad')
        else:
            continue
            
            #stem_word = stemmer.stem(word)  # stemming word
            #tweets_clean.append(stem_word)
    
    tweet = ' '.join(tweets_clean)

    return tweet


def test_lookup(func):
    freqs = {('sad', 0): 4,
             ('happy', 1): 12,
             ('oppressed', 0): 7}
    word = 'happy'
    label = 1
    if func(freqs, word, label) == 12:
        return 'SUCCESS!!'
    return 'Failed Sanity Check!'


def lookup(freqs, word, label):
    '''
    Input:
        freqs: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Output:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n



def normalize_sentiment(val):
    
    senMin = -1
    senMax = 1
    neutral = 0
    polMin = 1
    polMax =5


    newval = ((val -senMin)/(senMax-senMin))*(polMax-polMin)+polMin
        
    return round(newval)

def get_opinion(val):
    if val < 0:
        
        senti= 'Negative' if round(val)==0 else 'Very Negative' 

    elif val > 0:

        senti= 'Positive' if round(val)==0 else 'Very Positive'

    else:
        senti= 'Neutral'
    
    return senti
        

In [41]:
tweet = "Hey Buddy @kiran listen. Your work is :@ . Check his work at https://kiran"
print(process_tweet(tweet))
print(get_opinion(TextBlob(process_tweet(tweet)).sentiment.polarity))
print(TextBlob(process_tweet(tweet)).sentiment)

hey buddy listen work Sad check work
Negative
Sentiment(polarity=-0.5, subjectivity=1.0)


In [ ]:
#Aim is to get tweets based on hashtags which has covid-19 apps, security, covid19.
#Data Extraction first
#Reference : https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

In [42]:
access_token_key='1097367016456011776-bKEHl6bEgWc7gnd3cIqu3KlYsWgoi4'
access_token_secret='k8CVegeF8hzHJJQDTWEqQ1rzUVYn228TK5eMaO8Pan2M9'
consumer_key='CdNDp6wqPDmgUAQqCkikjNmTG'
consumer_secret='jTSjfniCLUEdmLplHGEniFVz0L2Ha7yfrWhxUvOGBWuN51OOyQ'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)

auth.set_access_token(access_token_key,access_token_secret)

In [43]:
twtr = tweepy.API(auth,wait_on_rate_limit=True)

In [131]:

#covid_keys = '#covid19apps OR #covidtracker OR #covidappsecurity OR #COVID19APPS OR #covid19tracker OR #covidappssecurity OR #Covid19AppSecurity OR #Corona19TrackerApps OR #Corona19Apps OR #Corona19AppsPrivacy OR #CoronaAppsSurveillance OR #Covid19Tracker OR #Covid19Surveillance -filter:retweets' #202,30,0,24
covid_keys = '#Covid19Tracker'


In [132]:
counter = 0
new_entry=[]
for page in tweepy.Cursor(twtr.search,q=covid_keys,include_rts=False,lang='en').pages(100):
    for status in page:
        #print("Page: ",counter)
        status = status._json
        #print(status['text'])
        original_twt = status['text']
        twt=process_tweet(original_twt)
        #opinion = polarities[normalize_sentiment(TextBlob(twt).sentiment.polarity)]
        sentiment = TextBlob(twt).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        opinion = get_opinion(polarity)
        timestamp = status['created_at']
        try:
            location = status['user']['location']
        except TypeError:
            location = ''
        
        
        counter+=1
        
        new_entry.append([counter,original_twt,location,timestamp,polarity,subjectivity,opinion])
        
        
    
 

In [133]:
len(new_entry)

83

In [ ]:
#Five categories: 
#{1 to be strongly negative, 2 to be negative, 3 to be neutral, 4 to be positive, and 5 to be strongly positive}

In [ ]:
# 1=5
# -1=1


#normalization formula from:https://stats.stackexchange.com/questions/281162/scale-a-number-between-a-range

    #   or
#   newvalue= (max'-min')/(max-min)*(value-min)+min'.

In [134]:
df = pd.DataFrame(new_entry, columns=['id','Tweet','location','time','polarity','subjectivity','opinion'])
#july_13

In [135]:
import numpy as np
np.unique(df['opinion'],return_counts=True)

(array(['Negative', 'Neutral', 'Positive'], dtype=object),
 array([13, 61,  9], dtype=int64))

In [137]:
df.to_csv('tweets_data_onlyCTracker.csv')

,id,Tweet,location,time,polarity,subjectivity,opinion
0,1,Join our WEBINAR Wed July 22 9:00 EDT as we di...,"Washington, DC",Mon Jul 20 21:03:00 +0000 2020,0.000000,0.0,Neutral
1,2,RT @WFIChampions: Join our WEBINAR Wed July 22...,"Vienna, Austria",Mon Jul 20 21:02:44 +0000 2020,0.000000,0.0,Neutral
2,3,RT @VertoAnalytics: The COVID-19 crisis has af...,New York City,Mon Jul 20 20:17:57 +0000 2020,0.333333,1.0,Positive
